# 🌽 Corn Diseases Detection - Edge Models Training

**Entrenamiento de 4 arquitecturas edge en Google Colab con GPU**

Este notebook entrena y compara:
- MobileNetV3Large
- EfficientNetLiteB2
- MobileViT (con fine-tuning)
- PMVT (con fine-tuning)

---

## ⚙️ Configuración Inicial

**IMPORTANTE:** Antes de ejecutar:
1. Runtime > Change runtime type > **GPU (T4)**
2. Sube tu carpeta `data/` a Google Drive en: `MyDrive/corn-diseases-data/`
3. Ejecuta todas las celdas en orden

## 1️⃣ Verificar GPU y Recursos

In [ ]:
# Verificar GPU disponible
!nvidia-smi

import tensorflow as tf
print(f"\n✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU disponible: {tf.config.list_physical_devices('GPU')}")

# Verificar memoria
!free -h

Fri Oct  3 03:22:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2️⃣ Clonar Repositorio

In [ ]:
# Clonar repositorio
!git clone https://github.com/ojgonzalezz/corn-diseases-detection.git
%cd corn-diseases-detection

# Verificar estructura
!ls -la

fatal: destination path 'corn-diseases-detection' already exists and is not an empty directory.
/content/corn-diseases-detection
total 80
drwxr-xr-x 9 root root  4096 Oct  3 03:21 .
drwxr-xr-x 1 root root  4096 Oct  3 02:05 ..
-rw-r--r-- 1 root root 12978 Oct  3 02:00 colab_edge_models_training.ipynb
drwxr-xr-x 5 root root  4096 Oct  3 03:18 data
drwxr-xr-x 4 root root  4096 Oct  3 02:00 experimentation
drwxr-xr-x 3 root root  4096 Oct  3 02:00 experiments
drwxr-xr-x 8 root root  4096 Oct  3 03:19 .git
-rw-r--r-- 1 root root   689 Oct  3 02:00 .gitignore
-rw-r--r-- 1 root root  1068 Oct  3 02:00 LICENSE
drwxr-xr-x 3 root root  4096 Oct  3 02:10 models
-rw-r--r-- 1 root root  1137 Oct  3 02:00 pyproject.toml
-rw-r--r-- 1 root root 11346 Oct  3 02:50 README.md
-rw-r--r-- 1 root root   722 Oct  3 02:50 requirements.txt
drwxr-xr-x 9 root root  4096 Oct  3 02:00 src
drwxr-xr-x 2 root root  4096 Oct  3 02:50 tests


## 3️⃣ Instalar Dependencias

In [ ]:
# Instalar dependencias
!pip install -q -r requirements.txt

# Verificar instalación
import tensorflow as tf
import keras
import mlflow
import numpy as np
import pandas as pd

print("✅ Todas las dependencias instaladas correctamente")
print(f"   TensorFlow: {tf.__version__}")
print(f"   Keras: {keras.__version__}")
print(f"   MLflow: {mlflow.__version__}")

✅ Todas las dependencias instaladas correctamente
   TensorFlow: 2.19.0
   Keras: 3.10.0
   MLflow: 3.3.2


## 4️⃣ Montar Google Drive y Copiar Datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive montado")
print("\nVerificando datos en Drive...")
!ls -la /content/drive/MyDrive/corn-diseases-data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Google Drive montado

Verificando datos en Drive...
total 12
drwx------ 6 root root 4096 Oct  3 01:40 test
drwx------ 6 root root 4096 Oct  3 01:40 train
drwx------ 6 root root 4096 Oct  3 01:40 val


In [ ]:
# Copiar datos desde Google Drive
import shutil
import os

# Ruta en Google Drive
drive_data_path = '/content/drive/MyDrive/corn-diseases-data'

# Copiar train, val, test
for split in ['train', 'val', 'test']:
    src = f'{drive_data_path}/{split}'
    dst = f'data/{split}'

    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"✅ Copiado: {split}")
    else:
        print(f"⚠️  No encontrado: {src}")

# Verificar estructura
print("\n📁 Estructura de datos:")
!tree data/ -L 2 || find data/ -maxdepth 2 -type d

✅ Copiado: train
✅ Copiado: val
✅ Copiado: test

📁 Estructura de datos:
/bin/bash: line 1: tree: command not found
data/
data/test
data/test/Healthy
data/test/Blight
data/test/Common_Rust
data/test/Gray_Leaf_Spot
data/val
data/val/Healthy
data/val/Blight
data/val/Common_Rust
data/val/Gray_Leaf_Spot
data/train
data/train/Healthy
data/train/Blight
data/train/Common_Rust
data/train/Gray_Leaf_Spot


## 5️⃣ Verificar Datos

In [ ]:
import os

def count_images(path):
    """Cuenta imágenes por clase."""
    counts = {}
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            counts[class_name] = len([f for f in os.listdir(class_path)
                                      if f.endswith(('.jpg', '.jpeg', '.png'))])
    return counts

print("📊 Distribución de datos:\n")
for split in ['train', 'val', 'test']:
    path = f'data/{split}'
    if os.path.exists(path):
        counts = count_images(path)
        total = sum(counts.values())
        print(f"{split.upper()}: {total} imágenes")
        for class_name, count in counts.items():
            print(f"  - {class_name}: {count}")
        print()
    else:
        print(f"⚠️  {split.upper()}: No encontrado\n")

📊 Distribución de datos:

TRAIN: 3856 imágenes
  - Healthy: 964
  - Blight: 964
  - Common_Rust: 964
  - Gray_Leaf_Spot: 964

VAL: 716 imágenes
  - Healthy: 174
  - Blight: 171
  - Common_Rust: 195
  - Gray_Leaf_Spot: 176

TEST: 722 imágenes
  - Healthy: 175
  - Blight: 173
  - Common_Rust: 197
  - Gray_Leaf_Spot: 177



## 6️⃣ Configurar Variables de Entorno

In [ ]:
# Configurar variables de entorno para TensorFlow
import os
os.chdir('/content/corn-diseases-detection')

# Pull últimos cambios
!git pull origin main

# Verificar commit actual
!git log --oneline -3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['PYTHONUNBUFFERED'] = '1'

# Configurar GPU memory growth
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth habilitado para {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)

print("✅ Configuración de entorno completada")

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 1.13 KiB | 385.00 KiB/s, done.
From https://github.com/ojgonzalezz/corn-diseases-detection
 * branch            main       -> FETCH_HEAD
   6f87329..e313c84  main       -> origin/main
Updating 6f87329..e313c84
Fast-forward
 experiments/edge_models/train_edge_model.py |  4 ++++
 src/utils/utils.py                          | 15 ++++++++++++---
 2 files changed, 16 insertions(+), 3 deletions(-)
e313c84 (HEAD -> main, origin/main, origin/HEAD) fix: corregir error de dimensiones en etiquetas del dataset
6f87329 feat: solucionar problema de memoria SIGKILL en entrenamiento edge models
b8a1f26 fix: reduce batch_size to 8 for Colab RAM constraints
✅ GPU memory growth habilitado para 1 GPU(s)
✅ Configuración de entorno completada


In [ ]:
# Verificar que batch_size es 16
!grep -n "batch_size.*8" experiments/edge_models/train_all_models.py

27:# NOTA: batch_size=8 para evitar OOM con 4056 imágenes en memoria
35:        'batch_size': 8,  # Reducido a 8 para evitar OOM en Colab
45:        'batch_size': 8,  # Reducido a 8 para evitar OOM en Colab
55:        'batch_size': 8,  # Reducido a 8 para evitar OOM en Colab
66:        'batch_size': 8,  # Reducido a 8 para evitar OOM en Colab


In [ ]:
# Limpiar memoria
import gc
import tensorflow as tf

tf.keras.backend.clear_session()
gc.collect()

print("✅ Memoria limpiada")
print("✅ Código actualizado")
print("✅ Listo para entrenar con batch_size=8")

✅ Memoria limpiada
✅ Código actualizado
✅ Listo para entrenar con batch_size=8


## 7️⃣ Entrenar Todos los Modelos Edge

**Esto tomará aproximadamente 2-3 horas con GPU T4**

In [ ]:
# Ejecutar entrenamiento de todos los modelos
!python experiments/edge_models/train_all_models.py

[2025-10-03 03:23:49] INFO [__main__] ======================================================================
[2025-10-03 03:23:49] INFO [__main__] =========== ENTRENAMIENTO DE TODAS LAS ARQUITECTURAS EDGE ===========
[2025-10-03 03:23:49] INFO [__main__] ======================================================================
[2025-10-03 03:23:49] INFO [__main__] Total de experimentos: 4
[2025-10-03 03:23:49] INFO [__main__] 
[2025-10-03 03:23:49] INFO [__main__] ======================================================================
[2025-10-03 03:23:49] INFO [__main__] ================= EXPERIMENTO 1/4: MobileNetV3Large =================
[2025-10-03 03:23:49] INFO [__main__] ======================================================================
[2025-10-03 03:23:49] INFO [__main__] Comando: python experiments/edge_models/train_edge_model.py --model MobileNetV3Large --lr 0.001 --dropout 0.3 --epochs 30 --batch-size 8
[2025-10-03 03:23:49] INFO [__main__] 
2025-10-03 03:23:51.579506: E ex

## 8️⃣ Comparar Resultados

In [ ]:
# Comparar modelos
!python experiments/edge_models/compare_models.py

[2025-10-03 04:15:59] INFO [__main__] ======================================================================
[2025-10-03 04:15:59] INFO [__main__] ================ ANÁLISIS COMPARATIVO DE MODELOS EDGE ================
[2025-10-03 04:15:59] INFO [__main__] ======================================================================
[2025-10-03 04:15:59] INFO [__main__] Cargando experimentos de MLflow...
[2025-10-03 04:15:59] INFO [__main__] Experimentos encontrados: 17
[2025-10-03 04:15:59] INFO [__main__] 
[2025-10-03 04:15:59] INFO [__main__] ======================================================================
[2025-10-03 04:15:59] INFO [__main__] ==================== COMPARACIÓN DE MODELOS EDGE ====================
[2025-10-03 04:15:59] INFO [__main__] ======================================================================
[2025-10-03 04:15:59] INFO [__main__] 
[2025-10-03 04:15:59] INFO [__main__] Tabla Comparativa:
[2025-10-03 04:15:59] INFO [__main__] ==================================

## 9️⃣ Seleccionar Mejor Modelo

In [ ]:
# Seleccionar mejor modelo
!python experiments/edge_models/select_best_model.py

[2025-10-03 04:16:12] INFO [__main__] ======================================================================
[2025-10-03 04:16:12] INFO [__main__] ================== SELECCIÓN DEL MEJOR MODELO EDGE ==================
[2025-10-03 04:16:12] INFO [__main__] ======================================================================
[2025-10-03 04:16:12] INFO [__main__] Cargando experimentos de MLflow...
[2025-10-03 04:16:12] INFO [__main__] Experimentos encontrados: 17
[2025-10-03 04:16:12] INFO [__main__] 
[2025-10-03 04:16:12] WARNING [__main__]   Ningún modelo cumple los requisitos mínimos
[2025-10-03 04:16:12] INFO [__main__] Usando el mejor modelo disponible sin filtro de requisitos
[2025-10-03 04:16:12] INFO [__main__] ======================================================================
[2025-10-03 04:16:12] INFO [__main__] ======================== MODELO SELECCIONADO ========================
[2025-10-03 04:16:12] INFO [__main__] ========================================================

## 🔟 Ver Resultados

In [ ]:
# Mostrar archivo de mejor modelo
import json

with open('experiments/edge_models/best_edge_model.json', 'r') as f:
    best_model = json.load(f)

print("🏆 MEJOR MODELO SELECCIONADO\n")
print(f"Modelo: {best_model['selected_model']['name']}")
print(f"\nMétricas:")
print(f"  Accuracy: {best_model['performance_metrics']['test_accuracy']:.2%}")
print(f"  Min Recall: {best_model['performance_metrics']['min_recall']:.2%}")
print(f"\nCaracterísticas:")
print(f"  Tamaño: {best_model['model_characteristics']['model_size_mb']} MB")
print(f"  Parámetros: {best_model['model_characteristics']['total_parameters']:,}")
print(f"  Edge-ready: {'✅ Sí' if best_model['model_characteristics']['suitable_for_edge'] else '❌ No'}")

# Mostrar comparación completa
import pandas as pd
df_comparison = pd.DataFrame(best_model['all_models_comparison'])
print("\n📊 COMPARACIÓN DE TODOS LOS MODELOS\n")
print(df_comparison.to_string(index=False))

🏆 MEJOR MODELO SELECCIONADO

Modelo: MobileNetV3Large

Métricas:
  Accuracy: 60.97%
  Min Recall: 34.29%

Características:
  Tamaño: 11.43 MB
  Parámetros: 2,996,352
  Edge-ready: ✅ Sí

📊 COMPARACIÓN DE TODOS LOS MODELOS

        model_name  test_accuracy  min_recall  size_mb  parameters  meets_requirements
  MobileNetV3Large         0.6097      0.3429    11.43     2996352               False
EfficientNetLiteB2         0.2736      0.0000    33.45     8769374               False
              PMVT         0.0000      0.0000    11.43     2996352               False
         MobileViT         0.0000      0.0000    11.43     2996352               False
              PMVT         0.0000      0.0000    11.43     2996352               False
         MobileViT         0.0000      0.0000    11.43     2996352               False
EfficientNetLiteB2         0.0000      0.0000    33.45     8769374               False
  MobileNetV3Large         0.0000      0.0000    11.43     2996352               F

## 1️⃣1️⃣ Descargar Resultados

In [ ]:
# Comprimir resultados para descargar
!zip -r edge_models_results.zip \
    experiments/edge_models/best_edge_model.json \
    experiments/edge_models/comparison_results.csv \
    models/exported/*.keras \
    models/exported/*.json \
    models/mlruns/

print("\n✅ Resultados comprimidos en: edge_models_results.zip")
print("\nPara descargar:")
print("  1. Click en el icono de carpeta (Files) a la izquierda")
print("  2. Busca 'edge_models_results.zip'")
print("  3. Click derecho > Download")

# También copiar a Google Drive
!cp edge_models_results.zip /content/drive/MyDrive/
print("\n✅ También guardado en Google Drive")

	zip warning: name not matched: models/exported/*.keras
	zip warning: name not matched: models/exported/*.json
  adding: experiments/edge_models/best_edge_model.json (deflated 82%)
  adding: experiments/edge_models/comparison_results.csv (deflated 64%)
  adding: models/mlruns/ (stored 0%)
  adding: models/mlruns/.trash/ (stored 0%)
  adding: models/mlruns/0/ (stored 0%)
  adding: models/mlruns/0/meta.yaml (deflated 27%)
  adding: models/mlruns/516230521730669656/ (stored 0%)
  adding: models/mlruns/516230521730669656/306f64ed1a7d4246817c4868dfd9c51b/ (stored 0%)
  adding: models/mlruns/516230521730669656/306f64ed1a7d4246817c4868dfd9c51b/tags/ (stored 0%)
  adding: models/mlruns/516230521730669656/306f64ed1a7d4246817c4868dfd9c51b/tags/mlflow.source.type (stored 0%)
  adding: models/mlruns/516230521730669656/306f64ed1a7d4246817c4868dfd9c51b/tags/mlflow.source.git.commit (stored 0%)
  adding: models/mlruns/516230521730669656/306f64ed1a7d4246817c4868dfd9c51b/tags/mlflow.user (stored 0%)
  

## 1️⃣2️⃣ Ver Experimentos en MLflow

In [ ]:
# Iniciar MLflow UI en Colab
import subprocess
import time
from pyngrok import ngrok
from google.colab import userdata
import os

# Ensure pyngrok is installed
try:
    import pyngrok
except ImportError:
    print("Installing pyngrok...")
    !pip install -q pyngrok

# Get your ngrok authtoken from the Colab secrets manager
# Ensure you have added NGROK_AUTH_TOKEN to your Colab secrets
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok authtoken loaded from Colab secrets.")
except Exception as e:
    print(f"⚠️ Could not load NGROK_AUTH_TOKEN from Colab secrets: {e}")
    print("Please add your ngrok authtoken to Colab secrets with the name 'NGROK_AUTH_TOKEN'.")
    print("You can get your authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken")
    # Set a dummy token to prevent ngrok.connect from failing immediately if token is missing
    # This will still result in an auth error, but allows the script to continue.
    # The user needs to fix the secret for it to work.
    ngrok.set_auth_token("dummy_token")


# Iniciar MLflow en background
# Check if MLflow is already running to avoid multiple processes
try:
    # This command lists processes, filtering for 'mlflow ui'
    # Adjusting for potential differences in process listing command output across environments
    # Using 'pgrep -f' which is generally reliable
    check_process = subprocess.run(['pgrep', '-f', 'mlflow ui'], capture_output=True, text=True)
    if check_process.stdout:
         print("MLflow UI process already running.")
         # Attempt to get the existing tunnel if available
         try:
             # Use ngrok.get_tunnels() to see if a tunnel is already active on port 5000
             active_tunnels = ngrok.get_tunnels()
             mlflow_tunnel = next((t for t in active_tunnels if t.proto == 'http' and t.config['addr'].endswith(':5000')), None)

             if mlflow_tunnel:
                 public_url = mlflow_tunnel.public_url
                 print(f"✅ MLflow UI already running. Tunnel URL: {public_url}")
                 start_new_process = False # Existing process and tunnel found
             else:
                 print("MLflow UI process running, but no active tunnel found on port 5000.")
                 print("Attempting to create a new tunnel.")
                 start_new_process = False # Process is running, just need to create tunnel

         except Exception as e:
             print(f"⚠️ Could not check for existing MLflow UI tunnel: {e}")
             print("Attempting to start a new process and tunnel.")
             start_new_process = True # If check fails, assume we need to start fresh

    else:
        print("MLflow UI process not found. Starting now...")
        start_new_process = True

    if start_new_process:
        mlflow_process = subprocess.Popen(
            ['mlflow', 'ui', '--host', '0.0.0.0', '--port', '5000',
             '--backend-store-uri', 'file:///content/corn-diseases-detection/models/mlruns'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        time.sleep(7) # Give MLflow UI a bit more time to start

    # Create public tunnel with ngrok (always attempt to create/re-create tunnel if not found)
    try:
        # Terminate existing tunnels to avoid conflicts
        ngrok.kill()
        print("Terminated existing ngrok tunnels.")

        public_url = ngrok.connect(5000).public_url
        print(f"✅ MLflow UI Tunnel: {public_url}")
        print("Click the link above to access the MLflow UI.")
    except Exception as e:
        print(f"❌ Failed to create ngrok tunnel: {e}")
        print("Please ensure your ngrok authtoken is correctly set in Colab secrets ('NGROK_AUTH_TOKEN') and try again.")
        print("If the issue persists, try restarting the Colab runtime ('Runtime' > 'Restart runtime').")


except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")


print("\nNote: The MLflow UI will run in the background. You can access it via the ngrok tunnel URL.")
print("If the tunnel URL is not displayed, check the cell output for errors or warnings.")

✅ ngrok authtoken loaded from Colab secrets.
MLflow UI process already running.
MLflow UI process running, but no active tunnel found on port 5000.
Attempting to create a new tunnel.
Terminated existing ngrok tunnels.
✅ MLflow UI Tunnel: https://unamused-nonequitably-vivien.ngrok-free.dev
Click the link above to access the MLflow UI.

Note: The MLflow UI will run in the background. You can access it via the ngrok tunnel URL.
If the tunnel URL is not displayed, check the cell output for errors or warnings.


## 1️⃣3️⃣ Limpiar

In [ ]:
# Liberar espacio si es necesario
# !rm -rf data/  # Eliminar datos (ya están en Drive)
# !rm -rf models/mlruns/  # Eliminar runs de MLflow (ya descargados)

print("💡 Descomenta las líneas anteriores si necesitas liberar espacio")

---

## ✅ Resumen

**Archivos generados:**
- `experiments/edge_models/best_edge_model.json` - Mejor modelo seleccionado
- `experiments/edge_models/comparison_results.csv` - Comparación completa
- `models/exported/*.keras` - Modelos entrenados
- `models/mlruns/` - Experimentos MLflow
- `edge_models_results.zip` - Todo comprimido

**Siguiente paso:**
1. Descarga `edge_models_results.zip`
2. Extrae en tu proyecto local
3. Usa el mejor modelo para deployment

---

**Desarrollado con ❤️ para Edge Computing**